Problem 1

In [1]:
import pandas as pd
import numpy as np
import zipfile as zp
import os
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
# Loading dataset using Pandas data frames
zip_path = "C:\\Users\\10476\\work2do\\Data Mining\\Homework 5\\ml-100k.zip"
file_path = "C:\\Users\\10476\\work2do\\Data Mining\\Homework 5\\ml-100k"

with zp.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(file_path)

data = os.path.join(file_path, 'u.data')
df = pd.read_csv(data, sep='\t', names=['user_id', 'item_id', 'rating', 'timestamp'])

df.head()

,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [3]:
# Converting the ratings data into a utility matrix
u_matrix = df.pivot(index='user_id', columns='item_id', values='rating')

u_matrix

item_id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.0,4.0,3.0,3.0,5.0,4.0,1.0,5.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
940,NaN,NaN,NaN,2.0,NaN,NaN,4.0,5.0,3.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
941,5.0,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# Finding the 10 most similar users for user 1 based on the consine similarity
user_means = u_matrix.mean(axis=1)
centered_matrix = u_matrix.sub(user_means, axis=0)
centered_matrix = centered_matrix.fillna(0)
#print(centered_matrix)

user1_vct = centered_matrix.loc[1].values.reshape(1, -1)
cos_sim = cosine_similarity(user1_vct, centered_matrix.values)[0]
df_sim = pd.DataFrame({'user_id':centered_matrix.index, 'similarity':cos_sim}).sort_values(by='similarity', ascending=False)

top10_similar = df_sim[df_sim['user_id'] != 1].head(10)

top10_similar

,user_id,similarity
772,773,0.204792
867,868,0.202321
591,592,0.196592
879,880,0.195801
428,429,0.190661
275,276,0.187476
915,916,0.186358
221,222,0.182415
456,457,0.182253
7,8,0.180891


In [5]:
# Finding the expected rating for item 508 for user 1
similar_user_ids = top10_similar['user_id'].values
item_508_ratings = u_matrix.loc[similar_user_ids, 508].dropna()

if not item_508_ratings.empty:
    expected_rating = item_508_ratings.mean()
    print('The expected rating for item 508 is', expected_rating)
else:
    print('No expected rating')

The expected rating for item 508 is 4.2


Problem 2

In [6]:
# Some libraries are leaded in Problem 1
from scipy.spatial.distance import euclidean

In [7]:
# u.data is loaded in Problem 1 as 'df'
df_rating = df

item = os.path.join(file_path, 'u.item')
df_movies = pd.read_csv(item, sep='|', encoding='latin-1',
                        names=['item_id', 'title', 'release_date', 'video_release_date', 'imdb_url'] + [f'genre_{i}' for i in range(19)],
                        usecols=range(24))

df_movies

,item_id,title,release_date,video_release_date,imdb_url,genre_0,genre_1,genre_2,genre_3,genre_4,...,genre_9,genre_10,genre_11,genre_12,genre_13,genre_14,genre_15,genre_16,genre_17,genre_18
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1677,1678,Mat' i syn (1997),06-Feb-1998,NaN,http://us.imdb.com/M/title-exact?Mat%27+i+syn+...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1678,1679,B. Monkey (1998),06-Feb-1998,NaN,http://us.imdb.com/M/title-exact?B%2E+Monkey+(...,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
1679,1680,Sliding Doors (1998),01-Jan-1998,NaN,http://us.imdb.com/Title?Sliding+Doors+(1998),0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1680,1681,You So Crazy (1994),01-Jan-1994,NaN,http://us.imdb.com/M/title-exact?You%20So%20Cr...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [21]:
# Building a user profile on centered data for both user 200 and 15
# Centered matrix for users is built in Problem 1 as 'centered_matrix'
centered_user_matrix = centered_matrix

user200_vct = centered_user_matrix.loc[200].values
user15_vct = centered_user_matrix.loc[15].values

movie95_raters = centered_matrix[95][centered_matrix[95] != 0].index
movie95_user_vct = centered_matrix.loc[movie95_raters].values
movie95_vct = np.mean(movie95_user_vct, axis=0)

print(f'Shape of user200_vct:{user200_vct.shape}')
print(f'Shape of user15_vct:{user15_vct.shape}')
print(f'Shape of movie95_vct:{movie95_vct.shape}')

Shape of user200_vct:(1682,)
Shape of user15_vct:(1682,)
Shape of movie95_vct:(1682,)


In [22]:
# Calculating the cosine similarity and distance between the user's preferences and the item 95
cos_sim_200 = cosine_similarity([user200_vct], [movie95_vct])[0][0]
cos_sim_15 = cosine_similarity([user15_vct], [movie95_vct])[0][0]

dist_200 = euclidean(user200_vct, movie95_vct)
dist_15 = euclidean(user15_vct, movie95_vct)

print(f'Cosine Similarity between user_200 and item_95:{cos_sim_200:.4f}')
print(f'Cosine Similarity between user_15 and item_95:{cos_sim_15:.4f}')
print(f'Distance between user_200 and item_95:{dist_200:.4f}')
print(f'Distance between user_15 and item_95:{dist_15:.4f}')

Cosine Similarity between user_200 and item_95:0.3650
Cosine Similarity between user_15 and item_95:0.1221
Distance between user_200 and item_95:11.7602
Distance between user_15 and item_95:13.6774


In [23]:
# Which user would a recommender system suggest this movie to?
if cos_sim_200 > cos_sim_15:
    print('Recommender system would suggest movie 95 to user 200 based on Cosine Similarity')
else:
    print('Recommender system would suggest movie 95 to user 15 based on Cosine Similarity')

if dist_200 < dist_15:
    print('Recommender system would suggest movie 95 to user 200 based on Euclidean Distance')
else:
    print('Recommender system would suggest movie 95 to user 15 based on Euclidean Distance')

Recommender system would suggest movie 95 to user 200 based on Cosine Similarity
Recommender system would suggest movie 95 to user 200 based on Euclidean Distance
